In [1]:
import numpy as np
from copy import deepcopy

import fiona
import shapely.geometry
from rtree.index import Index

In [2]:
%%time
geoms = []
properties = []
with fiona.open("../output/full-usa-3-13-2021_filtered.gpkg") as src:
    for i, row in enumerate(src):
        if i % 10000 == 0:
            print(i)
        geom = shapely.geometry.shape(row["geometry"])
        geoms.append(geom)
        properties.append(row["properties"])

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
CPU times: user 18.5 s, sys: 976 ms, total: 19.5 s
Wall time: 19.4 s


In [3]:
def do_filter(geoms, properties, idx):
    mask = np.zeros(len(geoms))

    valid_ids = []
    for i in range(len(geoms)):
        if mask[i] == 0:
            src_geom = geoms[i]
            result_idxs = []
            result_areas = []
            for j in idx.intersection(src_geom.bounds):
                target_geom = geoms[j]
                target_properties = properties[j]
                if src_geom.intersects(target_geom):
                    result_idxs.append(j)
                    result_areas.append(target_properties["area"])

            j = result_idxs[np.argmax(result_areas)] 
            valid_ids.append(j)

            mask[i] = 1
            mask[j] = 1
    return set(valid_ids)

In [4]:
valid_ids = set(range(len(geoms)))

t_geoms = geoms
t_properties = properties
i = 0
while True:
    
    t_geoms = [t_geoms[j] for j in range(len(t_geoms)) if j in valid_ids]
    t_properties = [t_properties[j] for j in range(len(t_properties)) if j in valid_ids]
        
    idx = Index()
    for j, geom in enumerate(t_geoms):
        idx.insert(j, geom.bounds)
        
    print(i, len(t_geoms))
    i+=1

    valid_ids = do_filter(t_geoms, t_properties, idx)
    
    if len(valid_ids) == len(t_geoms):
        break

0 424874
1 361244
2 360875
3 360857


In [8]:
with fiona.open("../output/full-usa-3-13-2021_filtered.gpkg") as src:
    with fiona.open(
        "../output/full-usa-3-13-2021_filtered_deduplicated.gpkg", "w",
        crs=src.crs, driver=src.driver, schema=src.schema
    ) as dst:
        for i in range(len(t_geoms)):
            if i % 10000 == 0:
                print(i)
                
            row = {
                "geometry": shapely.geometry.mapping(t_geoms[i]),
                "properties": t_properties[i]
            }
            dst.write(row)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
